In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()

from IPython.core.pylabtools import figsize

import statsmodels.api as sm

## _R 스타일 모형 정의_

In [2]:
from patsy import dmatrix

In [3]:
# 예제 데이터 만들기

np.random.seed(0)
x1 = np.random.rand(5) + 10
x2 = np.random.rand(5) * 10
y = x1 + 2 * x2 + np.random.randn(5)
df = pd.DataFrame(np.array([x1, x2, y]).T, columns=["x1", "x2", "y"])
df

,x1,x2,y
0,10.548814,6.458941,23.610739
1,10.715189,4.375872,20.921207
2,10.602763,8.917730,29.199261
3,10.544883,9.636628,29.939813
4,10.423655,3.834415,18.536348


In [4]:
dmatrix("x1", df)

DesignMatrix with shape (5, 2)
  Intercept        x1
          1  10.54881
          1  10.71519
          1  10.60276
          1  10.54488
          1  10.42365
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)

### R-style formula 연산자

In [5]:
# 상수항을 제외하려는 경우 : "- 1", "+ 0"

In [6]:
dmatrix("x1 - 1", df)

DesignMatrix with shape (5, 1)
        x1
  10.54881
  10.71519
  10.60276
  10.54488
  10.42365
  Terms:
    'x1' (column 0)

In [7]:
dmatrix("x1 + 0", df)

DesignMatrix with shape (5, 1)
        x1
  10.54881
  10.71519
  10.60276
  10.54488
  10.42365
  Terms:
    'x1' (column 0)

In [8]:
dmatrix("x1 + x2 - 1", df)

DesignMatrix with shape (5, 2)
        x1       x2
  10.54881  6.45894
  10.71519  4.37587
  10.60276  8.91773
  10.54488  9.63663
  10.42365  3.83442
  Terms:
    'x1' (column 0)
    'x2' (column 1)

In [9]:
# 두 변수의 곱을 새로운 변수로 추가하는 경우 : ":"

In [10]:
dmatrix("x1 + x2 + x1:x2 - 1", df)

DesignMatrix with shape (5, 3)
        x1       x2      x1:x2
  10.54881  6.45894   68.13417
  10.71519  4.37587   46.88830
  10.60276  8.91773   94.55258
  10.54488  9.63663  101.61711
  10.42365  3.83442   39.96862
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

In [11]:
dmatrix("x1 * x2 - 1", df)

DesignMatrix with shape (5, 3)
        x1       x2      x1:x2
  10.54881  6.45894   68.13417
  10.71519  4.37587   46.88830
  10.60276  8.91773   94.55258
  10.54488  9.63663  101.61711
  10.42365  3.83442   39.96862
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

In [12]:
# a / b = a + a:b

In [13]:
dmatrix("x1 / x2 -1", df)

DesignMatrix with shape (5, 2)
        x1      x1:x2
  10.54881   68.13417
  10.71519   46.88830
  10.60276   94.55258
  10.54488  101.61711
  10.42365   39.96862
  Terms:
    'x1' (column 0)
    'x1:x2' (column 1)

### 변환

- center(x) : 평균 제거
- standardize(x) : 평균 제거 및 표준편차로 스케일링
- scale(x) : standardize(x)와 같음

In [14]:
dmatrix("center(x1) + standardize(x1) + scale(x1)", df)

DesignMatrix with shape (5, 4)
  Intercept  center(x1)  standardize(x1)  scale(x1)
          1    -0.01825         -0.19319   -0.19319
          1     0.14813          1.56828    1.56828
          1     0.03570          0.37799    0.37799
          1    -0.02218         -0.23480   -0.23480
          1    -0.14341         -1.51828   -1.51828
  Terms:
    'Intercept' (column 0)
    'center(x1)' (column 1)
    'standardize(x1)' (column 2)
    'scale(x1)' (column 3)

### 변수 보호 

In [15]:
# 변수 보호를 하지 않았을 때

dmatrix("x1 + x2", df)

DesignMatrix with shape (5, 3)
  Intercept        x1       x2
          1  10.54881  6.45894
          1  10.71519  4.37587
          1  10.60276  8.91773
          1  10.54488  9.63663
          1  10.42365  3.83442
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)

In [16]:
# 변수 보호를 했을 때

dmatrix("I(x1 + x2)", df)

DesignMatrix with shape (5, 2)
  Intercept  I(x1 + x2)
          1    17.00775
          1    15.09106
          1    19.52049
          1    20.18151
          1    14.25807
  Terms:
    'Intercept' (column 0)
    'I(x1 + x2)' (column 1)

### 다항선형회귀

In [17]:
dmatrix("x1 + I(x1**2) + I(x1**3) + I(x1**4)")

DesignMatrix with shape (5, 5)
  Intercept        x1  I(x1 ** 2)  I(x1 ** 3)   I(x1 ** 4)
          1  10.54881   111.27747  1173.84524  12382.67452
          1  10.71519   114.81528  1230.26750  13182.54925
          1  10.60276   112.41859  1191.94772  12637.93965
          1  10.54488   111.19456  1172.53366  12364.23047
          1  10.42365   108.65258  1132.55698  11805.38301
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'I(x1 ** 2)' (column 2)
    'I(x1 ** 3)' (column 3)
    'I(x1 ** 4)' (column 4)

### 카테고리 변수 인코딩

- 카테고리 값을 이용하기 위한 사전 작업
    - full-rank-encoding
    - reduced-rank-encoding

In [18]:
df2 = pd.DataFrame(["A", "B", "C", "D"], columns=["x3"])
df2

,x3
0,A
1,B
2,C
3,D


In [22]:
# full-rank-encoding

dmatrix("x3 + 0", df2)

DesignMatrix with shape (4, 4)
  x3[A]  x3[B]  x3[C]  x3[D]
      1      0      0      0
      0      1      0      0
      0      0      1      0
      0      0      0      1
  Terms:
    'x3' (columns 0:4)

In [23]:
# reduced-rank-encoding

dmatrix("x3", df2)

DesignMatrix with shape (4, 4)
  Intercept  x3[T.B]  x3[T.C]  x3[T.D]
          1        0        0        0
          1        1        0        0
          1        0        1        0
          1        0        0        1
  Terms:
    'Intercept' (column 0)
    'x3' (columns 1:4)

### `OLS.from_formula` 메서드

In [24]:
np.random.seed(0)
x1 = np.random.rand(20) + 10
x2 = np.random.rand(20) * 10
y = x1 + 2 * x2 + np.random.randn(20)
df4 = pd.DataFrame(np.array([x1, x2, y]).T, columns=["x1", "x2", "y"])

In [26]:
# 직접 데이터 행렬을 만드는 경우

dfy = df4.iloc[:, -1]
dfX = sm.add_constant(df4.iloc[:,:-1])
model1 = sm.OLS(dfy, dfX)
print(model1.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     246.8
Date:                Sun, 01 Jul 2018   Prob (F-statistic):           2.75e-13
Time:                        23:36:41   Log-Likelihood:                -29.000
No. Observations:                  20   AIC:                             64.00
Df Residuals:                      17   BIC:                             66.99
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4226     10.140      0.140      0.8

In [27]:
# 모형 정의 문자열을 사용하는 경우

model2 = sm.OLS.from_formula("y ~ x1 + x2", data=df4)
print(model2.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     246.8
Date:                Sun, 01 Jul 2018   Prob (F-statistic):           2.75e-13
Time:                        23:37:38   Log-Likelihood:                -29.000
No. Observations:                  20   AIC:                             64.00
Df Residuals:                      17   BIC:                             66.99
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4226     10.140      0.140      0.8

## _입력변수가 카테고리값인 경우_

### 카테고리 독립 변수와 더미 변수

- 더미 변수의 예 1 : y가 카테고리 값을 가지는 변수 x1에만 의존하는 경우
    - `sm.OLS.from_formula("y ~ C(x1)", data=)`


- 더미 변수의 예 2 : y가 카테고리 값을 가지는 변수 x1과 실수 값을 가지는 변수 x2에 의존하는 경우
    - `sm.OLS.from_formula("y ~ C(x1) + x2", data=)`
    
       
- 더미 변수의 예 3 : y가 카테고리 값을 가지는 변수 x1과 실수 값을 가지는 변수 x2에 의존하는 경우. 이때, x2가 y에 미치는 영향력이 x1값에 따라서 달라진다.
    - `sm.OLS.from_formula("y ~ C(x1) + C(x1):x2", data=)`

### 보스턴 집값 데이터의 카테고리 변수

In [29]:
from sklearn.datasets import load_boston

boston = load_boston()
dfX0_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
dfX_boston = sm.add_constant(dfX0_boston)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
df_boston = pd.concat([dfX_boston, dfy_boston], axis=1)

In [30]:
model = sm.OLS(dfy_boston, dfX_boston)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sun, 01 Jul 2018   Prob (F-statistic):          6.95e-135
Time:                        23:47:49   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.4911      5.104      7.149      0.0

In [33]:
# reduced-rank-encoding

boston = load_boston()
dfX_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
df_boston = pd.concat([dfX_boston, dfy_boston], axis=1)
model = sm.OLS.from_formula("MEDV ~ CRIM + ZN + INDUS + C(CHAS) + NOX + RM + AGE + DIS + RAD + TAX + \
PTRATIO + B + LSTAT", data=df_boston)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sun, 01 Jul 2018   Prob (F-statistic):          6.95e-135
Time:                        23:50:13   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         36.4911      5.104      7.

In [34]:
# full-rank-encoding

boston = load_boston()
dfX_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
df_boston = pd.concat([dfX_boston, dfy_boston], axis=1)
model = sm.OLS.from_formula("MEDV ~ CRIM + ZN + INDUS + C(CHAS) + NOX + RM + AGE + DIS + RAD + TAX + \
PTRATIO + B + LSTAT + 0", data=df_boston)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sun, 01 Jul 2018   Prob (F-statistic):          6.95e-135
Time:                        23:50:36   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
C(CHAS)[0.0]    36.4911      5.104      7.149   